# Day 23 - Concurrency

* https://adventofcode.com/2019/day/23

We get to wire up 50 Intcode CPUs today. Clearly we need queues here! I could re-tool my Intcode module to use asyncio coroutines throughout, but we can also use threads. I'll use that here, via [`concurrency.futures()`](https://docs.python.org/3/library/concurrent.futures.html) to provide a ready-made threadpool, and the standard [`queue` module](https://docs.python.org/3/library/queue.html) to provide the 'network layer', which is simply a mapping with 50 queues (so I don't have to special-case address 255).

We need a way of shutting down the CPUs, however. I'm going to use special queue value for this, a sentinel that when received simply raises a halt exception. The simplest sentinel value in Python is the `None` object, which also makes it easy to type hint.

In [1]:
from __future__ import annotations
import logging
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from queue import Empty, SimpleQueue
from typing import (
    ContextManager,
    Generator,
    Iterator,
    List,
    Mapping,
    NamedTuple,
    Optional,
    TYPE_CHECKING
)

from intcode import CPU, Halt, Instruction, InstructionSet, base_opcodes


NETWORK_SIZE = 50


class Packet(NamedTuple):
    x: int
    y: int


class Network(ContextManager["Network"]):
    _queues: Mapping[int, SimpleQueue[Optional[Packet]]]
    
    def __init__(self):
        self._queues = {addr: SimpleQueue() for addr in range(NETWORK_SIZE)}
        self.broadcast = self._queues[255] = SimpleQueue()
    
    def __exit__(self, *exc) -> None:
        for q in self._queues.values():
            q.put(None)
    
    def __getitem__(self, addr: int) -> SimpleQueue[Optional[Packet]]:
        return self._queues[addr]
    
    def reader(self, addr: int) -> Iterator[int]:
        queue = self._queues[addr]
        while True:
            try:
                packet = queue.get(timeout=0.1)
            except Empty:
                yield -1
                continue
            if packet is None:
                raise Halt
            yield packet.x
            yield packet.y


NetworkDriver = Generator[None, int, None]


class NetworkCard(ContextManager[NetworkDriver]):
    address: int
    network: Network
    _runner: Optional[NetworkDriver] = None
        
    def __init__(self, address: int, network: Network) -> None:
        self.address = address
        self.network = network

    def powerdown(self) -> None:
        if self._runner is not None:
            self._runner.close()
            self._runner = None
    
    def __enter__(self) -> NetworkDriver:
        if self._runner is None:
            self._runner = self.run()
            # prime the runner, so it is waiting for input
            next(self._runner)
        return self._runner
    
    def __exit__(self, *exc) -> None:
        self.powerdown()
        
    def run(self) -> NetworkDriver:
        network = self.network
        while True:
            dest = yield None
            packet = Packet((yield None), (yield None))
            try:
                network[dest].put(packet)
            except KeyError:
                # unknown network destination
                pass
            
    def receive(self) -> Iterator[int]:
        yield self.address
        yield from self.network.reader(self.address)


def networked_intcode(memory: List[int], address: int, network: Network) -> None:
    networkcard = NetworkCard(address, network)
    with networkcard as driver:
        opcodes: InstructionSet = {
            **base_opcodes,
            3: Instruction(partial(next, networkcard.receive()), output=True),
            4: Instruction(driver.send, 1),
        }
        CPU(opcodes).reset(memory).execute()


def run_network(memory: List) -> int:
    with ThreadPoolExecutor(max_workers=NETWORK_SIZE) as executor:
        with Network() as network:
            machines = [
                executor.submit(networked_intcode, memory, addr, network)
                for addr in range(NETWORK_SIZE)
            ]
            packet = network.broadcast.get(timeout=15)
        
        for fut in machines:
            fut.cancel()
    
    return packet.y

In [2]:
import aocd
data = aocd.get_data(day=23, year=2019)
memory = list(map(int, data.split(',')))

In [3]:
print("Part 1:", run_network(memory))

Part 1: 26779


## Part 2, concurrency and synchronisation

We now need an extra thread that implements the NAT, which needs to know if the network is idle. This can be tricky with multiple threads all racing to send and receive. Note that we can't do this with the threading *Event* primitive, because checking on all 50 events still leaves room for race conditions (events we already checked could be set or cleared before we checked on all of them); we need a sort of reverse [barrier](https://en.wikipedia.org/wiki/Barrier_(computer_science)).

Since I already have a `Network` class, I just used a lock and a set of flags, and a sum of flags set (easily incremented or decremented as needed). The NAT can then just check the sum, if it is 50 all CPUs are idle.

In [4]:
from threading import Lock


class MonitoringNetwork(Network):
    _idle_flags: List[bool]
    _idle_lock: Lock
    _idle_level: int = 0
    idle: bool = False
    
    def __init__(self):
        super().__init__()
        self._idle_flags = [False] * NETWORK_SIZE
        self._idle_lock = Lock()
    
    def __exit__(self, *exc) -> None:
        for q in self._queues.values():
            q.put(None)
    
    def __getitem__(self, addr: int) -> SimpleQueue[Optional[Packet]]:
        return self._queues[addr]
    
    def reader(self, addr: int) -> Iterator[int]:
        queue = self._queues[addr]
        while True:
            try:
                packet = queue.get(timeout=0.1)
            except Empty:
                self._set_idle_flag(addr, True)
                yield -1
                continue
            if packet is None:
                raise Halt
            self._set_idle_flag(addr, False)
            yield packet.x
            yield packet.y
            
    def _set_idle_flag(self, addr: int, flag: bool) -> None:
        flags = self._idle_flags
        with self._idle_lock:
            if flags[addr] != flag:
                self._idle_level += 1 if flag else -1
                assert 0 <= self._idle_level <= 50
                self.idle = self._idle_level == 50
            flags[addr] = flag
            

def nat(network: MonitoringNetwork) -> int:
    queue = network.broadcast
    addr_zero = network[0]
    last_received: Optional[Packet] = None
    last_sent_y: Optional[int] = None

    while True:
        if last_received is not None and network.idle:
            addr_zero.put(last_received)
            if last_received.y == last_sent_y:
                return last_sent_y
            last_sent_y = last_received.y
        try:
            packet = queue.get(timeout=0.1)
            if packet is not None:
                last_received = packet
        except Empty:
            pass


def run_network_with_nat(memory: List) -> int:
    with ThreadPoolExecutor(max_workers=NETWORK_SIZE + 1) as executor:
        with MonitoringNetwork() as network:
            machines = [
                executor.submit(networked_intcode, memory, addr, network)
                for addr in range(NETWORK_SIZE)
            ]
            nat_fut = executor.submit(nat, network)
            result = nat_fut.result(timeout=30)
        
        for fut in machines:
            fut.cancel()
    
    return result

In [5]:
print("Part 2:", run_network_with_nat(memory))

Part 2: 19216
